In [1]:
# # Import API info
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [2]:
# Install dependencies
!pip install -q langchain-core requests langchain-google-genai google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15

In [3]:
# Import libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

**Tool Create**

In [4]:
@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [5]:
print(multiply.invoke({'a': 122, 'b': 361}))

44042


In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


**Tool Binding**

In [7]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash", google_api_key = api_key)

In [8]:
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--5ff8ad3d-54e1-4179-a67b-42f2cf4916fe-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [9]:
llm_with_tools = llm.bind_tools([multiply])

**Tool Calling**

In [10]:
llm_with_tools.invoke("Hi, how are you?")

AIMessage(content='I am doing well, thank you for asking. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--90e6e5f7-d39b-4a77-bc33-89308aaa63ef-0', usage_metadata={'input_tokens': 26, 'output_tokens': 18, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}})

In [11]:
llm_with_tools.invoke("can you multiply 100 by 50?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 100.0, "b": 50.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--e875d256-e804-4b1d-808d-d0705ff276e0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 100.0, 'b': 50.0}, 'id': '414502a5-edc7-4dcc-92b2-b592de91e002', 'type': 'tool_call'}], usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

In [12]:
llm_with_tools.invoke("can you multiply 100 by 50?").tool_calls

[{'name': 'multiply',
  'args': {'a': 100.0, 'b': 50.0},
  'id': '6f452efd-373e-40c2-86db-d87198fc3d54',
  'type': 'tool_call'}]

In [13]:
query = HumanMessage("can you multiply 100 by 50?")
messages = [query]

In [14]:
messages

[HumanMessage(content='can you multiply 100 by 50?', additional_kwargs={}, response_metadata={})]

In [15]:
result = llm_with_tools.invoke(messages)

In [16]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 100.0, "b": 50.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--a73c4102-73aa-40a9-9b46-b5cae3d78d76-0', tool_calls=[{'name': 'multiply', 'args': {'a': 100.0, 'b': 50.0}, 'id': '2c413018-9b47-4d35-a81d-6f8764e0a50d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

In [17]:
messages.append(result)

In [18]:
messages

[HumanMessage(content='can you multiply 100 by 50?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 100.0, "b": 50.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--a73c4102-73aa-40a9-9b46-b5cae3d78d76-0', tool_calls=[{'name': 'multiply', 'args': {'a': 100.0, 'b': 50.0}, 'id': '2c413018-9b47-4d35-a81d-6f8764e0a50d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})]

**Tool Execution**

In [19]:
tool_result = multiply.invoke(result.tool_calls[0])

In [20]:
tool_result

ToolMessage(content='5000', name='multiply', tool_call_id='2c413018-9b47-4d35-a81d-6f8764e0a50d')

In [21]:
messages.append(tool_result)
messages

[HumanMessage(content='can you multiply 100 by 50?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 100.0, "b": 50.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--a73c4102-73aa-40a9-9b46-b5cae3d78d76-0', tool_calls=[{'name': 'multiply', 'args': {'a': 100.0, 'b': 50.0}, 'id': '2c413018-9b47-4d35-a81d-6f8764e0a50d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='5000', name='multiply', tool_call_id='2c413018-9b47-4d35-a81d-6f8764e0a50d')]

In [22]:
llm_with_tools.invoke(messages).content

'The product of 100 and 50 is 5000.'

**Currency Coversion Tool**

In [23]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated, get_type_hints

In [24]:
# Create Tool

@tool
def currency_conversion_rate(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion rates between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [25]:
print(currency_conversion_rate.args)
print(convert.args)

{'base_currency': {'title': 'Base Currency', 'type': 'string'}, 'target_currency': {'title': 'Target Currency', 'type': 'string'}}
{'base_currency_value': {'title': 'Base Currency Value', 'type': 'number'}, 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}


In [26]:
currency_conversion_rate.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756425601,
 'time_last_update_utc': 'Fri, 29 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756512001,
 'time_next_update_utc': 'Sat, 30 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.6276}

In [27]:
convert.invoke({'base_currency_value': 10, 'conversion_rate':87.6276})

876.2760000000001

In [34]:
# Tool Binding

llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash', google_api_key = api_key)
llm_with_tools = llm.bind_tools([currency_conversion_rate, convert])

In [51]:
print(llm_with_tools.kwargs["tools"])

[{'type': 'function', 'function': {'name': 'currency_conversion_rate', 'description': 'This function fetches the currency conversion rates between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculates the target currency value from a given base currency value', 'parameters': {'properties': {'base_currency_value': {'type': 'number'}}, 'required': ['base_currency_value'], 'type': 'object'}}}]


In [59]:
messages = [HumanMessage('What is the conversion rate between USD and INR, and based on that can you convert 10 USD to INR')]

In [60]:
messages

[HumanMessage(content='What is the conversion rate between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [61]:
ai_message = llm_with_tools.invoke(messages)

In [62]:
messages.append(ai_message)

In [63]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_conversion_rate', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b4b2040a-137f-4e40-acfd-be33161c73a8-0', tool_calls=[{'name': 'currency_conversion_rate', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '965de0ac-34ab-456c-833d-30c87c214787', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 13, 'total_tokens': 104, 'input_token_details': {'cache_read': 0}})

In [64]:
ai_message.tool_calls

[{'name': 'currency_conversion_rate',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '965de0ac-34ab-456c-833d-30c87c214787',
  'type': 'tool_call'}]

In [74]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'currency_conversion_rate':
    tool_message1 = currency_conversion_rate.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [75]:
messages

[HumanMessage(content='What is the conversion rate between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_conversion_rate', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b4b2040a-137f-4e40-acfd-be33161c73a8-0', tool_calls=[{'name': 'currency_conversion_rate', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '965de0ac-34ab-456c-833d-30c87c214787', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 13, 'total_tokens': 104, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756

In [76]:
llm_with_tools.invoke(messages).content

'The conversion rate between USD and INR is 87.6276. I will now convert 10 USD to INR.'

**Using AgentExecutor**

In [86]:
from langchain.agents import initialize_agent, AgentType

# Initialize the Agent
agent_executor = initialize_agent(
    tools=[currency_conversion_rate, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [87]:
# --- Step 6: Run the Agent ---
user_query = "What is the conversion rate between USD and INR, and based on that can you convert 2950 USD to INR?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "currency_conversion_rate",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "INR"
  }
}
```
Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1756425601, 'time_last_update_utc': 'Fri, 29 Aug 2025 00:00:01 +0000', 'time_next_update_unix': 1756512001, 'time_next_update_utc': 'Sat, 30 Aug 2025 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 87.6276}
Thought:I have the conversion rate between USD and INR. Now I can convert 2950 USD to INR.
Action:
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 2950,
    "conversion_rate": 87.6276
  }
}
```
Observation: 258501.42
Thought:I have successfully converted 2950 USD to INR.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The conversion rate between USD and INR is 87

In [88]:
print(response['output'])

The conversion rate between USD and INR is 87.6276. Based on this rate, 2950 USD is equivalent to 258501.42 INR.
